### Парсинг и подготовка корпуса

In [3]:
from bs4 import BeautifulSoup
import requests
import re
from tqdm import tqdm
import pandas as pd

Генерируем ссылки на страницы с отзывами

In [4]:
links = ['https://www.banki.ru/services/responses/bank/tcs/?is_countable=on']

In [5]:
for i in range(2, 20):
    links.append('https://www.banki.ru/services/responses/bank/tcs/?is_countable=on&page={}&isMobile=0'.format(i))

Генерируем ссылки на страницы с отзывами для тестовой выборки

In [6]:
test_links = []
for i in range(21, 30):
    test_links.append('https://www.banki.ru/services/responses/bank/tcs/?is_countable=on&page={}&isMobile=0'.format(i))

Собираем ссылки на страницы с отзывами

In [7]:
good_links = []
bad_links = []
for l in tqdm(links):
    res = requests.get(l)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    reviews = soup.find_all('article', class_='responses__item')
    for r in reviews:
        rating = r.find('div', class_='flexbox flexbox--inline flexbox--row flexbox--gap_xsmall flexbox--align-items_baseline')
        spans = rating.find_all('span')
        rating = spans[1].text
        tag = r.find('a', class_='header-h3', href=True)
        if int(rating) > 3:
            good_links.append('https://www.banki.ru{}'.format(tag.get('href')))
        elif int(rating) < 3:
            bad_links.append('https://www.banki.ru{}'.format(tag.get('href')))

100%|██████████| 19/19 [00:27<00:00,  1.43s/it]


Собираем ссылки на страницы с отзывами для тестовой выборки

In [8]:
good_test_links = []
bad_test_links = []
for tl in tqdm(test_links):
    res = requests.get(tl)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    reviews = soup.find_all('article', class_='responses__item')
    for r in reviews:
        rating = r.find('div', class_='flexbox flexbox--inline flexbox--row flexbox--gap_xsmall flexbox--align-items_baseline')
        spans = rating.find_all('span')
        rating = spans[1].text
        tag = r.find('a', class_='header-h3', href=True)
        if int(rating) > 3:
            good_test_links.append('https://www.banki.ru{}'.format(tag.get('href')))
        elif int(rating) < 3:
            bad_test_links.append('https://www.banki.ru{}'.format(tag.get('href')))

100%|██████████| 9/9 [00:13<00:00,  1.55s/it]


Создаем корпуса с текстами хороших и плохих отзывов:

In [ ]:
with open('bad_texts.txt', 'w', encoding='utf-8') as f:
    f.write('')
with open('good_texts.txt', 'w', encoding='utf-8') as ff:
    ff.write('')

In [ ]:
print(len(good_links), len(bad_links))

262 201


In [ ]:
for gl in tqdm(good_links):
    res = requests.get(gl)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    rev = soup.find('div', class_='article-text response-page__text markup-inside-small markup-inside-small--bullet')
    with open('good_texts.txt', 'a', encoding='utf-8') as file:
        file.write(rev.text)

for bl in tqdm(bad_links):
    res = requests.get(bl)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    rev = soup.find('div', class_='article-text response-page__text markup-inside-small markup-inside-small--bullet')
    with open('bad_texts.txt', 'a', encoding='utf-8') as file:
        file.write(rev.text)

100%|██████████| 201/201 [01:19<00:00,  2.54it/s]


Создаем датафрейм для тестовой выборки

In [9]:
df_links = []
df_reviews = []
df_polarities = []

In [10]:
good_test_reviews = []
for gtl in tqdm(good_test_links):
    res = requests.get(gtl)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    rev = soup.find('div', class_='article-text response-page__text markup-inside-small markup-inside-small--bullet')
    good_test_reviews.append(rev.text)

bad_test_reviews = []        
for btl in tqdm(bad_test_links):
    res = requests.get(btl)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    rev = soup.find('div', class_='article-text response-page__text markup-inside-small markup-inside-small--bullet')
    bad_test_reviews.append(rev.text)

100%|██████████| 76/76 [01:10<00:00,  1.08it/s]


In [11]:
for gtl in range(len(good_test_links)):
    df_links.append(good_test_links[gtl])
    df_polarities.append('good')
    df_reviews.append(good_test_reviews[gtl])

for btl in range(len(bad_test_links)):
    df_links.append(bad_test_links[btl])
    df_polarities.append('bad')
    df_reviews.append(bad_test_reviews[btl])

In [12]:
df = pd.DataFrame({
    'review_link': df_links,
    'review_text': df_reviews,
    'polarity': df_polarities
})

In [13]:
df.head(5)

,review_link,review_text,polarity
0,https://www.banki.ru/services/responses/bank/r...,\n\t\tДобрый день.19.08.2021 через сайт Блабла...,good
1,https://www.banki.ru/services/responses/bank/r...,\n\t\tВ Тинькофф банке мне нравится большое ко...,good
2,https://www.banki.ru/services/responses/bank/r...,"\n\t\tПользуюсь тинькофф уже больше года , все...",good
3,https://www.banki.ru/services/responses/bank/r...,\n\t\tДоброго дня! Хотел бы поделиться опытом ...,good
4,https://www.banki.ru/services/responses/bank/r...,"\n\t\tЯвляюсь клиентом банка более пяти лет, в...",good


In [14]:
df = df.sample(frac=1).reset_index(drop=True)

In [15]:
df.head(5)

,review_link,review_text,polarity
0,https://www.banki.ru/services/responses/bank/r...,\n\t\tПо моему мнению один из лучших банков. П...,good
1,https://www.banki.ru/services/responses/bank/r...,\n\t\tХочу выразить благодарность своему персо...,good
2,https://www.banki.ru/services/responses/bank/r...,\n\t\tАктивировал спецпредложение 30% на товар...,good
3,https://www.banki.ru/services/responses/bank/r...,\n\t\tУвидел рекламу Тинькофф о карте с беспла...,good
4,https://www.banki.ru/services/responses/bank/r...,\n\t\tСегодня хотел уточнить многие нюансы по ...,good


In [18]:
from nltk import word_tokenize
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [19]:
with open('good_texts.txt', 'r', encoding='utf-8') as file:
    good_corpus = file.read()
    
with open('bad_texts.txt', 'r', encoding='utf-8') as file:
    bad_corpus = file.read()   

Создаем функцию, отвечающую за препроцессинг (токенизирует, приводит к нижнему регистру и приводит слова в начальную форму)

In [20]:
def preprocess(text):
    text = text.lower()
    words = word_tokenize(text)
    out = []
    for w in words:
        p = morph.parse(w)[0]
        out.append(p.normal_form)
    return out

In [24]:
preprocessed_good = preprocess(good_corpus)
preprocessed_bad = preprocess(bad_corpus)

In [ ]:
preprocessed_good[:10]

['спасибо',
 'банк',
 'за',
 'лояльность',
 'и',
 'оперативный',
 'решение',
 'вопрос',
 '!',
 'решить']

Убираем из корпусов слова, встретившиеся меньше четырех раз и создаем множества с уникальными для корпусов словами

In [25]:
from collections import Counter

In [26]:
good_c = Counter(preprocessed_good)
bad_c = Counter(preprocessed_bad)

In [27]:
good_c.most_common(10)

[(',', 3251),
 ('.', 1933),
 ('и', 1349),
 ('в', 1328),
 ('я', 1144),
 ('банк', 828),
 ('на', 693),
 ('не', 663),
 ('что', 537),
 ('с', 531)]

In [28]:
good_corp = []
for k, v in good_c.items():
    if v > 3:
        good_corp.append(k)
        
bad_corp = []
for k, v in bad_c.items():
    if v > 3:
        bad_corp.append(k)

In [29]:
len(good_corp), len(bad_corp)

(1174, 1290)

In [30]:
good_corp = set(good_corp)
bad_corp = set(bad_corp)

In [31]:
unique_b = bad_corp - good_corp
unique_g = good_corp - bad_corp

In [32]:
len(unique_b), len(unique_g)

(480, 364)

### **Используем шаблоны для улучшения работы функции**

Для улучшения точности модели попробуем учитывать н-граммы следующих форматов:
1. не + глагол (думаю, отрицания с глаголом будут совершенно разные в корпусе положительных и отрицательных отзывов; "не получилось" и "не разочаровался", например)
2. прилагательное + банк (поскольку мой корпус - отзывы на банки, мы сможем смотреть эпитеты, использующиеся по отношению к банку; "ужаснейший банк" и "отличный банк" явно помогут нам определиться с тональностью)
3. глагол + числительное (опять же, в связи со спецификой моего корпуса, часто будут встречаться сочетания типа "потерял 2000" или "перевел 500")

In [34]:
import stanza
stanza.download('ru')

2021-10-02 19:43:53 INFO: Downloading default packages for language: ru (Russian)...


2021-10-02 19:45:52 INFO: Finished downloading models and saved to /root/stanza_resources.


In [36]:
nlp = stanza.Pipeline(lang='RU', processors='tokenize,pos')
doc_good = nlp(good_corpus)
doc_bad = nlp(bad_corpus)

2021-10-02 19:47:10 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |

2021-10-02 19:47:10 INFO: Use device: cpu
2021-10-02 19:47:10 INFO: Loading: tokenize
2021-10-02 19:47:10 INFO: Loading: pos
2021-10-02 19:47:10 INFO: Done loading processors!


In [37]:
good_tagged = []
for sent in doc_good.sentences:
    for word in sent.words:
        good_tagged.append((word.text, word.upos))

bad_tagged = []
for sent in doc_bad.sentences:
    for word in sent.words:
        bad_tagged.append((word.text, word.upos))

In [50]:
def chuncker(corpus):
    bigrams = []
    for c in range(len(corpus) - 1):
        if corpus[c][0].lower() == 'не' and corpus[c+1][1] == 'VERB':
            bigrams.append((corpus[c][0].lower(), corpus[c+1][0]))
        if corpus[c][1] == 'ADJ' and corpus[c+1][0] == 'банк':
            bigrams.append((corpus[c][0].lower(), corpus[c+1][0]))
        if corpus[c][1] == 'VERB' and corpus[c+1][1] == 'NUM':
            bigrams.append((corpus[c][0].lower(), corpus[c+1][0]))
    return bigrams

In [64]:
good_bigrams = chuncker(good_tagged)
bad_bigrams = chuncker(bad_tagged)

In [65]:
good_bigrams[:10]

[('решили', 'меньше'),
 ('не', 'задумывался'),
 ('пишет', '1'),
 ('нашла', 'несколько'),
 ('не', 'подтверждаю'),
 ('не', 'отвечает'),
 ('не', 'вернём'),
 ('не', 'подтвердил'),
 ('опережает', 'многие'),
 ('не', 'встречал')]

Так же, как и с отдельными словами, отбираем только биграммы, встретившиеся в корпусе более 2 раз и уникальные для этого корпуса

In [66]:
good_c_bigrams = Counter(good_bigrams)

bad_c_bigrams = Counter(bad_bigrams)

In [69]:
good_corp_bigrams = []
for k, v in good_c_bigrams.items():
    if v > 2:
        good_corp_bigrams.append(k)
        
bad_corp_bigrams = []
for k, v in bad_c_bigrams.items():
    if v > 2:
        bad_corp_bigrams.append(k)

In [70]:
len(good_corp_bigrams), len(bad_corp_bigrams)

(28, 65)

In [71]:
good_corp_bigrams = set(good_corp_bigrams)
bad_corp_bigrams = set(bad_corp_bigrams)

In [72]:
unique_b_bigrams = bad_corp_bigrams - good_corp_bigrams
unique_g_bigrams = good_corp_bigrams - bad_corp_bigrams

In [75]:
len(unique_g_bigrams), len(unique_b_bigrams)

(17, 54)

In [74]:
unique_b_extended = unique_b | unique_b_bigrams
unique_g_extended = unique_g | unique_g_bigrams

In [76]:
len(unique_g_extended), len(unique_b_extended)

(381, 534)

Создаем функцию, определяющую полярность отзыва (положительный / отрицательный)

In [78]:
def guess(review):
    doc_review = nlp(good_corpus)
    review_tagged = []
    for sent in doc_review.sentences:
        for word in sent.words:
            review_tagged.append((word.text, word.upos))
    relevant_bigrams = chuncker(review_tagged)
    relevant_bigrams = set(relevant_bigrams)

    preprocessed_r = preprocess(review)
    review_words = set(preprocessed_r)
    
    review_relevant = review_words | relevant_bigrams
    bad_intersection = review_relevant & unique_b_extended
    good_intersection = review_relevant & unique_g_extended
    if len(bad_intersection) > len(good_intersection):
        return 'bad'
    elif len(good_intersection) > len(bad_intersection):
        return 'good'
    else:
        return 'unsure'

Проверяем работу функции

In [ ]:
predictions = []
accuracy = []
for i in range(len(df['review_text'])):
    prediction = guess(df['review_text'][i])
    predictions.append(prediction)
    if df['polarity'][i] == prediction:
        accuracy.append(1)
    else:
        accuracy.append(0)

In [ ]:
df['predictions'] = predictions
df['accuracy'] = accuracy

In [ ]:
df.head(5)

,review_link,review_text,polarity,predictions,accuracy
0,https://www.banki.ru/services/responses/bank/r...,"\n\t\tПользуясь многими банками в стране, хоте...",good,good,1
1,https://www.banki.ru/services/responses/bank/r...,"\n\t\tПользуюсь банком уже 3 года, за все врем...",good,good,1
2,https://www.banki.ru/services/responses/bank/r...,\n\t\tНачну с небольшой предыстории. Клиентом ...,good,good,1
3,https://www.banki.ru/services/responses/bank/r...,"\n\t\t12.08 оплатил полис ОСАГО, авторизовалис...",bad,bad,1
4,https://www.banki.ru/services/responses/bank/r...,\n\t\t12 августа пытался воспользоваться банко...,bad,bad,1


In [ ]:
accuracy_df = df['accuracy'].value_counts()

In [ ]:
accuracy = accuracy_df[1] / (accuracy_df[0] + accuracy_df[1])

In [ ]:
accuracy

0.8482142857142857

Итак, accuracy модели получилось приблизительно 0,85.

Для улучшения программы можно:
1. Учитывать частотность лексем, а не только факт пересечения с тем или иным множеством
2. Учитывать длину отзыва (хорошие отзывы обычно короче, чем плохие)
3. Брать только отзывы с одной звездой для корпуса негативных отзывов и только отзывы с пятью звездами для корпуса позитивных (так они будут более эмоционально заряженными). 